In [ ]:
import pandas as pd
from time import sleep
import requests
from dotenv import load_dotenv
import os

In [ ]:
# Load environment variables
load_dotenv()

headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.7',
    'Authorization': 'Bearer null',
    'Connection': 'keep-alive',
    'Referer': 'https://salute.regione.veneto.it/ivgStatistiche/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-GPC': '1',
    'User-Agent': os.getenv('USER_AGENT'),
    'dnt': '1',
    'sec-ch-ua': f'"Brave";v="{os.getenv("BROWSER_VERSION")}", "Chromium";v="{os.getenv("BROWSER_VERSION")}", "Not_A Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': f'"{os.getenv("PLATFORM")}"',
}

In [ ]:
aziende_ids = list(range(101, 123)) + list(range(501, 510)) + [901, 912]

anni_search = [1,2,3,4,5,6,7,8]

# Initialize empty lists to store the data
nomi = []
valori = []
anni = []

for anno in anni_search: 
  for aziende_id in aziende_ids:
    params = {
      'annoA': anno,
      'annoDa': anno,
      'idAzienda': aziende_id,
      'idTipoGrafico': '8',
      'isCategoriaEnable': 'false',
      'isCategoriaMax1': 'false',
    }

    response = requests.get('https://salute.regione.veneto.it/ivgStatistiche/api/grafico', params=params, headers=headers)
    data = response.json()

    if 'graficoTortaDto' not in data or data['graficoTortaDto'] is None:
        print(f"Skipping azienda {aziende_id} for year {anno} - no data found")
        continue

    aziende = data['graficoTortaDto']
    for azienda in aziende:
      nome = azienda['name']
      valore = azienda['value']
      print(f"{anno} - IVG {nome}: {valore}")

      # Append each nome and valore to their respective lists
      nomi.append(nome)
      valori.append(valore)
      anni.append(anno)

    sleep(1)

# Create the DataFrame after the loop is complete
df = pd.DataFrame({
    'nome_azienda': nomi,
    'valore': valori,
    'anno': anni
})

In [ ]:
anni_mapping = {
    1: 2015,
    2: 2016,
    3: 2017,
    4: 2018,
    5: 2019,
    6: 2020,
    7: 2021,
    8: 2022
}
df['anno_id'] = df['anno']
df['anno'] = df['anno_id'].map(anni_mapping)
df['valore'] = df['valore'].astype(int)

In [ ]:
df.to_csv('output/ivg_effettuate.csv', index=False, encoding='UTF-8')